# Comparing Elevation Measurements

Once you have an area of interest with multiple overlapping DEMs and/or laser altimetry measurements, you can quickly compare elevation values. `coincident` supports a main plotting panel for elevation comparisons, as well as dynamic boxplots and histograms for elevation differences over various terrain characteristics and land cover types. 

This notebook highlights the following functions:

* coincident.plot.compare_dems
* coincident.plot.boxplot_slope
* coincident.plot.boxplot_elevation
* coincident.plot.boxplot_aspect
* coincident.plot.hist_esa

```{warning}
Plotting routines are currently only tested for comparing datasets in EPSG:7912 (ITRF 2014)
```

In [ ]:
import coincident
import geopandas as gpd
from shapely.geometry import box
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
from rasterio.enums import Resampling

In [ ]:
%matplotlib inline
# %config InlineBackend.figure_format = 'retina'

## Load reference DEM

In [ ]:
workunit = "CO_WestCentral_2019"
df_wesm = coincident.search.wesm.read_wesm_csv()
gf_lidar = coincident.search.wesm.load_by_fid(
    df_wesm[df_wesm.workunit == workunit].index
)

gf_lidar

In [ ]:
# We will examine the 'sourcedem' 1m
# NOTE: it's important to note the *vertical* CRS of the data
gf_lidar.iloc[0]

In [ ]:
# NOTE: since we're going to compare to satellite altimetry, let's convert first to EPSG:7912 !
# GDAL / PROJ will use the following transform by default
!projinfo -s EPSG:6350+5703 -t EPSG:7912 -q -o proj

Now, let's grab the vendor DEM products for two arbitrary tiles of this flight to look at elevation differences between the aerial LIDAR derved DEM, Copernicus GLO-30 DEM, ICESat-2 elevations, and GEDI elevations.

In [ ]:
# We will rely on GDAL to do this reprojection
# gdalvsi = "/vsicurl?empty_dir=yes&url="
gdalvsi = "/vsicurl/"
url_tile_1 = f"{gdalvsi}https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/OPR/Projects/CO_WestCentral_2019_A19/CO_WestCentral_2019/TIFF/USGS_OPR_CO_WestCentral_2019_A19_be_w1032n1780.tif"
url_tile_2 = f"{gdalvsi}https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/OPR/Projects/CO_WestCentral_2019_A19/CO_WestCentral_2019/TIFF/USGS_OPR_CO_WestCentral_2019_A19_be_w1032n1779.tif"

# First build a mosaic in native 3D CRS
!gdalbuildvrt -q -a_srs EPSG:6350+5703 /tmp/mosaic.vrt '{url_tile_1}' '{url_tile_2}'

In [ ]:
# Then reproject
!GDAL_DISABLE_READDIR_ON_OPEN=EMPTY_DIR gdalwarp -q -overwrite -t_srs EPSG:7912 -r bilinear /tmp/mosaic.vrt /tmp/mosaic_7912.tif

In [ ]:
# Open our 'reference' DEM
da_lidar = rxr.open_rasterio("/tmp/mosaic_7912.tif", masked=True).squeeze()
da_lidar

In [ ]:
# Plot a single DEM
ax = coincident.plot.plot_dem(da_lidar, title="USGS_OPR_CO_WestCentral_2019_A19");

## Load secondary DEMs

```{note}
Currently coincident facilitates loading select global DEMs in EPSG:7912, if your dataset is not in that CRS be sure to first reproject your data
```

In [ ]:
# search geodataframe to grab the COP30 DEM
bbox_tile = da_lidar.rio.bounds()
poly_tile = box(*bbox_tile)
gf_tile = gpd.GeoDataFrame(geometry=[poly_tile], crs=da_lidar.rio.crs)

In [ ]:
# NOTE: this is 30m data, but we are going to resample it to 1m to match our lidar
da_cop = coincident.io.xarray.load_dem_7912("cop30", aoi=gf_tile)
da_cop

In [ ]:
ds_cop_r = (
    da_cop.rio.reproject_match(
        da_lidar,
        resampling=Resampling.bilinear,
    )
    .where(da_lidar.notnull())
    .to_dataset(name="elevation")
)

ds_cop_r

In [ ]:
# Create hillshade variables for plot backgrounds
# This function expects Datasets
ds_lidar = da_lidar.to_dataset(name="elevation")

ds_lidar["hillshade"] = coincident.plot.gdaldem(
    ds_lidar.elevation, "hillshade", scale=111120
)
ds_cop_r["hillshade"] = coincident.plot.gdaldem(
    ds_cop_r.elevation, "hillshade", scale=111120
)

In [ ]:
# Plot a single DEM with hillshade
ax = coincident.plot.plot_dem(
    ds_cop_r["elevation"],
    title="Resampled Copernicus DEM",
    da_hillshade=ds_lidar["hillshade"],
)

In [ ]:
# Get GEDI
gf_gedi = coincident.search.search(
    dataset="gedi",
    intersects=gf_tile,
)
data_gedi = coincident.io.sliderule.subset_gedi02a(
    gf_gedi, gf_tile, include_worldcover=True
)
# Get ICESAT-2
gf_is2 = coincident.search.search(
    dataset="icesat-2",
    intersects=gf_tile,
)
data_is2 = coincident.io.sliderule.subset_atl06(
    gf_is2, gf_tile, include_worldcover=True
)

In [ ]:
# NOTE: this works because ICESat-2 and GEDI are EPSG:7912
coincident.plot.plot_altimeter_points(
    data_is2, "h_li", da_hillshade=ds_lidar["hillshade"], title="ICESat-2 ATL06 points"
);

### Main Elevation Comparison Panel

'coincident' supports a main `compare_dems` function that allows you to compare multiple DEMs and altimetry points in a single panel. This function dynamically adjusts based on the number of datasets provided.

```{note}
`compare_dems` assumes that all DEMs are in the same coordinate reference system (CRS) and aligned. It also assumes that each DEM has an 'elevation' variable. If add_hillshade=True, it assumes the first DEM passed has a 'hillshade' variable, where the 'hillshade' variable can be calculated with `coincident.plot.create_hillshade` as seen below. A minimum of two DEMs must be provided and a maximum of 5 total datasets (datasets being DEMs and GeoDataFrames of altimetry points) can be provided.
```

In [ ]:
# list of DEMs to compare, where the first DEM in the list is the 'source' DEM
dem_list = [ds_lidar, ds_cop_r]
# dictionary of GeoDataFrames with altimetry points to compare
# where the key is the name/plot title of the dataset and the value is a tuple of the respective
# GeoDataFrame and the column name of the elevation values of interest
dicts_gds = {"ICESat-2": (data_is2, "h_li"), "GEDI": (data_gedi, "elevation_lm")}

axd = coincident.plot.compare_dems(
    dem_list,
    dicts_gds,
    # ds_wc=ds_wc,
    add_hillshade=True,
    diff_clim=(-5, 5),  # difference colormap limits
)

### Dynamic Elevation Difference Boxplots Based on Terrain Variables (slope, elevation, aspect)

`coincident` allows users to dynamically plot elevation differences based on grouped data such as slope values. In these functions (that are wrappers for the generalized coincident.plot.boxplot_terrain_diff function), the user provides a reference DEM and one other DEM or GeoDataFrame with elevation values to compare to. These function assumes that the reference DEM has the respective terrain variable (slope, aspect, etc.) of interest.

```{note}
Boxplots will only be generated with groups of value counts greater than 30. Dynamic ylims will be fit to the IQRs of the boxplots.
```

In [ ]:
# These routines assume rectilinear data, so we first convert to UTM
crs_utm = ds_lidar.rio.estimate_utm_crs()
ds_lidar_utm = ds_lidar.elevation.rio.reproject(crs_utm).to_dataset()
ds_cop_utm = ds_cop_r.elevation.rio.reproject(crs_utm).to_dataset()

In [ ]:
slope = coincident.plot.utils.gdaldem(ds_lidar_utm.elevation, "slope")
ds_lidar_utm["slope"] = slope

In [ ]:
_, ax = plt.subplots()
ds_lidar_utm.slope.plot.imshow(ax=ax)
ax.set_title("USGS_OPR_CO_WestCentral_2019_A19");

In [ ]:
ax_boxplot_dems = coincident.plot.boxplot_slope([ds_lidar_utm, ds_cop_utm])

In [ ]:
# we also support elevation differences based on elevation bins
# grouped on the reference DEM's elevation values
ax_elev = coincident.plot.boxplot_elevation(
    [ds_lidar_utm, ds_cop_utm], elevation_bins=np.arange(1700, 2000, 20)
)

In [ ]:
# lastly for the boxplots, let's look at aspect
aspect = coincident.plot.utils.gdaldem(ds_lidar_utm.elevation, "aspect")
ds_lidar_utm["aspect"] = aspect

In [ ]:
_, ax = plt.subplots()
ds_lidar_utm.aspect.plot.imshow(ax=ax)
ax.set_title("USGS_OPR_CO_WestCentral_2019_A19");

In [ ]:
ax_aspect = coincident.plot.boxplot_aspect([ds_lidar_utm, ds_cop_utm])

### Histograms of Elevation Differences over Land Cover

Plot elevation differences between DEMs or point data, grouped by ESA World Cover 2020 land cover class

```{note}
Histograms will only be generated with groups of value counts greater than a user-defined threshold (default is 30).
```

In [ ]:
ax_lulc_cop30 = coincident.plot.hist_esa([ds_lidar, ds_cop_r])